## PhoREAL / SlideRule Example

Demonstrate running the PhoREAL algorithm in SlideRule to produce canopy metrics over the Grand Mesa, Colorado region.

#### Imports

In [ ]:
import matplotlib.pyplot as plt
import matplotlib
import geopandas
import sliderule
from sliderule import icesat2

#### Initialize Client
* Organization currently set to "utexas"; if you want to be a member of the utexas SlideRule organization, make a request through the SlideRule provisioning system (https://ps.slideruleearth.io); otherwise, remove the organization parameter to default to the public SlideRule cluster.
* Notebook only processes one granule, so one desired_node is sufficient

In [ ]:
icesat2.init("slideruleearth.io", verbose=False, organization="utexas", desired_nodes=1)

#### Processing parameters
* 100m segments stepped every 100m
* Subsetted to the Grand Mesa region
* Time range is one day, Nov 14, 2019
* Only processing ground, canopy, and top of canopy photons
* Running PhoREAL algorithm using a binsize of 1m, and geolocating each segment at the center of the segment
* Sending reconstructed waveforms along with metrics (for diagnostics and demonstration purposes only)

In [ ]:
parms = {
    "poly": sliderule.toregion('grandmesa.geojson')['poly'],
    "t0": '2019-11-14T00:00:00Z',
    "t1": '2019-11-15T00:00:00Z',
    "srt": icesat2.SRT_LAND,
    "len": 100,
    "res": 100,
    "pass_invalid": True, 
    "atl08_class": ["atl08_ground", "atl08_canopy", "atl08_top_of_canopy"],
    "phoreal": {"binsize": 1.0, "geoloc": "center", "use_abs_h": False, "send_waveform": True}
}

#### Make Atl08 Request

In [ ]:
atl08 = icesat2.atl08p(parms, asset="icesat2", keep_id=True)

#### Print Resulting GeoDataFrame

In [ ]:
atl08

#### Plot Canopy Height

In [ ]:
canopy_gt1l = atl08[atl08['gt'] == icesat2.GT1L]
canopy_gt1l.plot.scatter(x='distance', y='h_canopy')

#### Plot Landcover

In [ ]:
atl08.plot('landcover')

#### Create and Plot 75th percentile Across All Ground Tracks

In [ ]:
atl08['75'] = atl08.apply(lambda row : row["canopy_h_metrics"][icesat2.P['75']], axis = 1)
atl08.plot.scatter(x='distance', y='75')

#### Create Sample Waveform Plots

In [ ]:
num_plots = 5
waveform_index = [96, 97, 98, 100, 101]
fig,ax = plt.subplots(num=1, ncols=num_plots, sharey=True, figsize=(12, 6))
for x in range(num_plots):
    ax[x].plot([x for x in range(len(canopy_gt1l['waveform'][waveform_index[x]]))], canopy_gt1l['waveform'][waveform_index[x]], zorder=1, linewidth=1.0, color='mediumseagreen')
plt.show()

#### Make Atl06 Request
* Below we run an ATL06-SR processing request on the same source data using the same parameters.  Because the `keep_id` argument is set to true here and above when we made the ATL08 request, we can merge the resulting dataframes and have a single table of both elevation data using the customized ATL06-SR algorithm, and vegatation data using the PhoREAL algorithm.

In [ ]:
atl06 = icesat2.atl06p(parms, asset="icesat2", keep_id=True)

#### Merge Atl06 and Atl08 GeoDataFrames

In [ ]:
gdf = geopandas.pd.merge(atl08, atl06, on='extent_id', how='left', suffixes=('.atl08','.atl06')).set_axis(atl08.index)
gdf